In [ ]:
import numpy as np
import uproot as uproot
import uproot3 as uproot3
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

from tqdm.notebook import tqdm

from get_MA_trio_cov_mat import get_MA_trio_cov_mat_pred, extract_trio

import pickle


In [ ]:
nuwro_fake_data_true_MA = 1.03 # From Afro slack 2024_06_10
genie_v2_fake_data_true_MA = 0.99 # Nitish slack 2023_04_25


In [ ]:
plt.rcParams.update({'font.size': 16})
s = 20

def plot_bold_MA_value(true_MA, extracted_MA, extracted_MA_sigma, label, color, zorder):
    plt.errorbar(true_MA, extracted_MA, yerr=extracted_MA_sigma, fmt="o", markersize=6, c=color, capthick=2, elinewidth=2, capsize=5, zorder=zorder, 
            label=label + f": {extracted_MA:.3f} "r"$\pm$"f" {extracted_MA_sigma:.3f} "
            r"$\mathrm{GeV/c^2}$"f"\n\t\t\tTrue: {true_MA:.3f} "r"$\mathrm{GeV/c^2}$""\n\t\t\t"
            fr"$\sigma = {np.abs(true_MA - extracted_MA)/extracted_MA_sigma:.3f}$")
    plt.errorbar(true_MA, extracted_MA, yerr=extracted_MA_sigma, fmt="o", markersize=8, c="k", capthick=4, elinewidth=4, capsize=6, zorder=9)

bins = np.linspace(-5, 5, 21)
bin_width = bins[1] - bins[0]

x = np.linspace(-5, 5, 1000)


# NuWro M_A Reweighting

In [ ]:
def get_nuwro_weight(true_Enu, true_Q2, true_MA, reweighting_ratios):
    """
    Get the reweighting factor for a given true neutrino energy, Q2, and MA value
    
    Parameters:
    -----------
    true_Enu : float
        True neutrino energy in GeV
    true_Q2 : float
        True Q2 in GeV^2
    true_MA : float
        True MA value in GeV (must be one of: 0.7, 0.8, 0.9, 1.0, 1.03, 1.1, 1.2, 1.3, 1.4, 1.5)
    reweighting_ratios : dict
        Dictionary containing the reweighting ratios and bins
    """
    # Check if MA value exists in the ratios
    if true_MA not in reweighting_ratios:
        raise ValueError(f"MA value {true_MA} not found in reweighting ratios. Available values: {sorted(reweighting_ratios.keys())}")
    
    # Get the bins for Enu and Q2
    q2_bins = reweighting_ratios[true_MA]['q2_bins']
    Enu_bins = reweighting_ratios[true_MA]['Enu_bins']
    
    # Get bin indices
    q2_idx = np.clip(np.digitize(true_Q2 / 1e6, q2_bins) - 1, 0, len(q2_bins)-2)
    Enu_idx = np.clip(np.digitize(true_Enu / 1e3, Enu_bins) - 1, 0, len(Enu_bins)-2)
    
    # Get the weight directly from the ratios
    weight = reweighting_ratios[true_MA]['ratios'][q2_idx, Enu_idx]
    
    return weight

print("loading nuwro fake data events for reweighting")

bdt_vars = [
    "nue_score",
    "numu_score",
    "numu_cc_flag"
]

eval_vars = [
    "run",
    "subrun",
    "event",
    "match_isFC",

    # these aren't present in the fake data file
    #"truth_nuEnergy",
    #"truth_nuPdg",
    #"truth_isCC",
    #"truth_vtxInside",
    #"match_completeness_energy",
    #"truth_energyInside",
    #"weight_cv",
    #"weight_spline",
]

eval_data_vars = [
    "match_isFC",
]

kine_vars = [
    "kine_reco_Enu",
]

pf_vars = [
    "reco_muonMomentum",
    #"truth_muonMomentum",
]

pf_data_vars = [
    "reco_muonMomentum",
]

loc = "/Users/leehagaman/data/processed_checkout_rootfiles/"

# not using run 1, that's numuCC only
"""
f = uproot3.open(loc + "checkout_fakedata_nuwro_run1.root")["wcpselection"] # run 1 is numuCC only
f_bdt = f["T_BDTvars"].pandas.df(bdt_vars, flatten=False)
f_eval = f["T_eval"].pandas.df(eval_vars, flatten=False)
f_kine = f["T_KINEvars"].pandas.df(kine_vars, flatten=False)
f_pfeval = f["T_PFeval"].pandas.df(pf_vars, flatten=False)
nuwro_fake_run1_vars_pot = np.sum(f["T_pot"].pandas.df("pot_tor875good", flatten=False)["pot_tor875good"].to_numpy())
nuwro_fake_run1_df = pd.concat([f_bdt, f_eval, f_kine, f_pfeval], axis=1, sort=False)
nuwro_fake_run1_df["file"] = "nuwro_fake_run1"
"""

f = uproot3.open(loc + "checkout_fakedata_nuwro_run2.root")["wcpselection"]
f_bdt = f["T_BDTvars"].pandas.df(bdt_vars, flatten=False)
f_eval = f["T_eval"].pandas.df(eval_vars, flatten=False)
f_kine = f["T_KINEvars"].pandas.df(kine_vars, flatten=False)
f_pfeval = f["T_PFeval"].pandas.df(pf_vars, flatten=False)
nuwro_fake_run2_vars_pot = np.sum(f["T_pot"].pandas.df("pot_tor875good", flatten=False)["pot_tor875good"].to_numpy())
nuwro_fake_run2_df = pd.concat([f_bdt, f_eval, f_kine, f_pfeval], axis=1, sort=False)
nuwro_fake_run2_df["file"] = "nuwro_fake_run2"

f = uproot3.open(loc + "checkout_fakedata_nuwro_run3.root")["wcpselection"]
f_bdt = f["T_BDTvars"].pandas.df(bdt_vars, flatten=False)
f_eval = f["T_eval"].pandas.df(eval_vars, flatten=False)
f_kine = f["T_KINEvars"].pandas.df(kine_vars, flatten=False)
f_pfeval = f["T_PFeval"].pandas.df(pf_vars, flatten=False)
nuwro_fake_run3_vars_pot = np.sum(f["T_pot"].pandas.df("pot_tor875good", flatten=False)["pot_tor875good"].to_numpy())
nuwro_fake_run3_df = pd.concat([f_bdt, f_eval, f_kine, f_pfeval], axis=1, sort=False)
nuwro_fake_run3_df["file"] = "nuwro_fake_run3"
del f
del f_bdt
del f_eval
del f_kine
del f_pfeval

total_wc_nuwro_fake_pot =  nuwro_fake_run2_vars_pot + nuwro_fake_run3_vars_pot
num_wc_nuwro_fake_events = len(nuwro_fake_run2_df) + len(nuwro_fake_run3_df)

wc_nuwro_fake_df = pd.concat([nuwro_fake_run2_df, nuwro_fake_run3_df], axis=0, sort=False)

print("adding reco_costheta and reco_muon_momentum to NuWro fake data events")

costheta_vals = []
muonmomentum_vals = []
reco_muonmomentum_x = wc_nuwro_fake_df["reco_muonMomentum[0]"].to_numpy()
reco_muonmomentum_y = wc_nuwro_fake_df["reco_muonMomentum[1]"].to_numpy()
reco_muonmomentum_z = wc_nuwro_fake_df["reco_muonMomentum[2]"].to_numpy()
reco_muonmomentum_t = wc_nuwro_fake_df["reco_muonMomentum[3]"].to_numpy()
for i in range(len(reco_muonmomentum_x)):
    if reco_muonmomentum_t[i] < 105.66 / 1000.: # surprising that this happens for positive values, but I did find some events
        costheta_vals.append(-1)
        muonmomentum_vals.append(-1)
    else:
        costheta_vals.append(reco_muonmomentum_z[i] / np.sqrt(reco_muonmomentum_x[i]**2 + reco_muonmomentum_y[i]**2 + reco_muonmomentum_z[i]**2))
        muon_KE = reco_muonmomentum_t[i] * 1000. - 105.66
        muonmomentum_vals.append(np.sqrt(muon_KE**2 + 2 * muon_KE * 105.66))
wc_nuwro_fake_df["reco_costheta"] = costheta_vals
wc_nuwro_fake_df["reco_muon_momentum"] = muonmomentum_vals

print("loading PeLEE NuWro fake data events for adding truth information")

for r in [2,3]:
    f = uproot.open(f"/Users/leehagaman/data/pelee_files/high_stat_prodgenie_bnb_nu_overlay_DetVar_Run{r}_NuWro_reco2_reco2.root")["nuselection"]["NeutrinoSelectionFilter"]
    run = f["run"].array()
    subrun = f["sub"].array()
    event = f["evt"].array()
    nu_pdg = f["nu_pdg"].array()
    true_NC = f["ccnc"].array()
    # https://internal.dunescience.org/doxygen/classgenie_1_1ScatteringType.html, equivalent to truth_nuScatType in WC files
    # different from truth_nuIntType, which comes from https://internal.dunescience.org/doxygen/MCNeutrino_8h_source.html but doesn't totally make sense with the values we see
    interaction = f["interaction"].array()
    true_nu_energy = f["nu_e"].array() * 1000.
    true_lep_energy = f["lep_e"].array() * 1000.
    true_lep_theta = f["theta"].array()
    true_inFV = f["isVtxInFiducial"].array()
    true_nu_vtx_x = f["true_nu_vtx_x"].array()
    true_nu_vtx_y = f["true_nu_vtx_y"].array()
    true_nu_vtx_z = f["true_nu_vtx_z"].array()
    if r == 2:
        print(f"creating df with {len(run)} entries")
        pelee_nuWro_truth_df = pd.DataFrame({"run": run, "subrun": subrun, "event": event, 
                                            "pl_true_nu_pdg": nu_pdg, "pl_true_NC": true_NC,
                                            "pl_true_nu_energy": true_nu_energy, "pl_true_lep_energy": true_lep_energy, "pl_true_lep_theta": true_lep_theta,
                                            "pl_interaction_type": interaction, "pl_inFV": true_inFV,
                                            "pl_true_nu_vtx_x": true_nu_vtx_x, "pl_true_nu_vtx_y": true_nu_vtx_y, "pl_true_nu_vtx_z": true_nu_vtx_z})
    else:
        print(f"appending df with {len(run)} entries")
        pelee_nuWro_truth_df = pd.concat([pelee_nuWro_truth_df, pd.DataFrame({"run": run, "subrun": subrun, "event": event, 
                                                                            "pl_true_nu_pdg": nu_pdg, "pl_true_NC": true_NC,
                                                                            "pl_true_nu_energy": true_nu_energy, "pl_true_lep_energy": true_lep_energy, "pl_true_lep_theta": true_lep_theta,
                                                                            "pl_interaction_type": interaction, "pl_inFV": true_inFV,
                                                                            "pl_true_nu_vtx_x": true_nu_vtx_x, "pl_true_nu_vtx_y": true_nu_vtx_y, "pl_true_nu_vtx_z": true_nu_vtx_z})])
        
# Q^2 = - (p_nu - p_mu)^2
#     = -p_nu^2 + 2 p_nu * p_mu - p_mu^2
#     = -m_nu^2 + 2 E_nu * E_mu - 2 * P_nu * P_mu * cos(theta) - m_mu^2
#     = 2 E_nu * E_mu - 2 * E_nu * P_mu * cos(theta) - m_mu^2
#     = 2 * E_nu * (E_mu - P_mu * cos(theta)) - m_mu^2

pelee_nuWro_truth_df["pl_true_lep_mass"] = [105.6583755 if abs(pdg) == 13 else 0.51099895069 if abs(pdg) == 11 else 0. for pdg in pelee_nuWro_truth_df["pl_true_nu_pdg"]]
pelee_nuWro_truth_df["pl_true_lep_KE"] = pelee_nuWro_truth_df["pl_true_lep_energy"] - pelee_nuWro_truth_df["pl_true_lep_mass"]
pelee_nuWro_truth_df["pl_true_lep_momentum"] = np.sqrt(pelee_nuWro_truth_df["pl_true_lep_energy"]**2 - pelee_nuWro_truth_df["pl_true_lep_mass"]**2)
pelee_nuWro_truth_df["pl_true_lep_costheta"] = np.cos(pelee_nuWro_truth_df["pl_true_lep_theta"])
pelee_nuWro_truth_df["pl_true_Q2"] = 2 * pelee_nuWro_truth_df["pl_true_nu_energy"] * (pelee_nuWro_truth_df["pl_true_lep_energy"] - pelee_nuWro_truth_df["pl_true_lep_momentum"] * pelee_nuWro_truth_df["pl_true_lep_costheta"]) - pelee_nuWro_truth_df["pl_true_lep_mass"]**2

print("merging truth information into NuWro fake data events")

nuwro_fake_df = pd.merge(wc_nuwro_fake_df, pelee_nuWro_truth_df, on=["run", "subrun", "event"], how="inner")
#nuwro_fake_pot = total_wc_nuwro_fake_pot * len(nuwro_fake_df) / num_wc_nuwro_fake_events

print("splitting into events that we are and aren't reweighting")

nuwro_numuCCinFV_fake_df = nuwro_fake_df.query("pl_true_nu_pdg == 14 and pl_true_NC == 0 and pl_inFV == 1")
nuwro_not_numuCCinFV_fake_df = nuwro_fake_df.query("not (pl_true_nu_pdg == 14 and pl_true_NC == 0 and pl_inFV == 1)")

print("getting M_A reweighting factors")

nuwro_fake_data_true_Enus = nuwro_numuCCinFV_fake_df["pl_true_nu_energy"].to_numpy()
nuwro_fake_data_true_Q2s = nuwro_numuCCinFV_fake_df["pl_true_Q2"].to_numpy()


In [ ]:
with open('nuwro_reweighting_ratios.pkl', 'rb') as f:
    reweighting_ratios = pickle.load(f)

MA_reweighted_nuwro_fake_data_dic = {}

rw_MA_vals = [0.7, 0.8, 0.9, 1.0, 1.03, 1.1, 1.2, 1.3, 1.4, 1.5]

for reweight_nuwro_M_A in [0.7, 0.8, 0.9, 1.0, 1.03, 1.1, 1.2, 1.3, 1.4, 1.5]:

    nuwro_fake_data_weights = []
    for i in range(len(nuwro_fake_data_true_Enus)):
        nuwro_fake_data_weights.append(get_nuwro_weight(nuwro_fake_data_true_Enus[i], nuwro_fake_data_true_Q2s[i], reweight_nuwro_M_A, reweighting_ratios))

    nuwro_not_numuCCinFV_fake_df["MA_reweight"] = 1.

    nuwro_fake_data_weights = np.clip(nuwro_fake_data_weights, 0, 10)

    nuwro_numuCCinFV_fake_df["MA_reweight"] = nuwro_fake_data_weights

    print("recombining NuWro fake data events")

    nuwro_fake_df = pd.concat([nuwro_not_numuCCinFV_fake_df, nuwro_numuCCinFV_fake_df], axis=0, sort=False)

    print("applying WC numuCC selection")

    nuwro_fake_sel_df = nuwro_fake_df.query("numu_cc_flag >= 0 and numu_score > 0.9 and nue_score < 7 and reco_muon_momentum>0")

    print("getting NuWro fake data histogram")

    data = []

    for containment in ["FC", "PC"]:
        
        if containment == "FC":
            containment_df = nuwro_fake_sel_df.query("match_isFC==1")
        else:
            containment_df = nuwro_fake_sel_df.query("match_isFC==0")
            
        for Enu_bin in range(4):
            
            if Enu_bin == 0:
                Enu_df = containment_df.query("200 < kine_reco_Enu <= 705")
            elif Enu_bin == 1:
                Enu_df = containment_df.query("705 < kine_reco_Enu < 1050")
            elif Enu_bin == 2:
                Enu_df = containment_df.query("1050 < kine_reco_Enu < 1570")
            elif Enu_bin == 3:
                Enu_df = containment_df.query("1570 < kine_reco_Enu < 4000")
            
            for theta_bin in range(9):
                
                if theta_bin == 0:
                    theta_df = Enu_df.query("-1 < reco_costheta <= -0.5")
                elif theta_bin == 1:
                    theta_df = Enu_df.query("-0.5 < reco_costheta <= 0.")
                elif theta_bin == 2:
                    theta_df = Enu_df.query("0. < reco_costheta <= 0.27")
                elif theta_bin == 3:
                    theta_df = Enu_df.query("0.27 < reco_costheta <= 0.45")
                elif theta_bin == 4:
                    theta_df = Enu_df.query("0.45 < reco_costheta <= 0.62")
                elif theta_bin == 5:
                    theta_df = Enu_df.query("0.62 < reco_costheta <= 0.76")
                elif theta_bin == 6:
                    theta_df = Enu_df.query("0.76 < reco_costheta <= 0.86")
                elif theta_bin == 7:
                    theta_df = Enu_df.query("0.86 < reco_costheta <= 0.94")
                else:
                    theta_df = Enu_df.query("0.94 < reco_costheta <= 1.")
                
                data += list(np.histogram(theta_df["reco_muon_momentum"].to_numpy(), 
                                            weights=theta_df["MA_reweight"].to_numpy(),
                                            bins = [i*100 for i in range(16)] + [1e9] # fifteen bins from 0 to 1500 plus an overflow
                                            )[0])
                
    print("done reweighting NuWro fake data, data array updated")

    MA_reweighted_nuwro_fake_data_dic[reweight_nuwro_M_A] = data


# Extracted M_A Tests

In [ ]:
filename = "plots/fake_data_nuwro_MA_reweight"

plt.figure(figsize=(10,7))
plt.plot([0, 2], [0, 2], c="k", ls="dashed", lw=2, zorder=5)

cov_MA, pred_MA, data, multisim_xs_MA_cov, xs_vars = get_MA_trio_cov_mat_pred(
    data_type = "NuWro",
)
constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(cov_MA, pred_MA, data)
extracted_MA = prior_removed_MA
extracted_MA_sigma = prior_removed_MA_sigma
plot_bold_MA_value(nuwro_fake_data_true_MA, extracted_MA, extracted_MA_sigma, "NuWro fake data", "k", 10)

for rw_true_MA in rw_MA_vals:
    reweighted_data = MA_reweighted_nuwro_fake_data_dic[rw_true_MA]
    constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(cov_MA, pred_MA, reweighted_data)
    extracted_MA = prior_removed_MA
    extracted_MA_sigma = prior_removed_MA_sigma
    plot_bold_MA_value(rw_true_MA, extracted_MA, extracted_MA_sigma, r"NuWro fake data with reweighted $M_A$", "tab:brown", 10)

plt.legend(loc="upper left", fontsize=6)
plt.xlabel(r"True $M_A$ (GeV/c$^2$)")
plt.ylabel(r"Reco $M_A$ (GeV/c$^2$)")
plt.xlim(0.6, 1.6)
plt.ylim(0.6, 1.6)
plt.title(r"NuWro fake data with reweighted $M_A$")
plt.savefig(filename + ".pdf")

prior_MA = 1.1
prior_MA_sigma = 0.1
plt.figure(figsize=(10,7))
plt.plot([0, 2], [0, 2], c="k", ls="dashed", lw=2, zorder=5)
plt.fill_between([0, 2], [prior_MA - prior_MA_sigma, prior_MA - prior_MA_sigma], [prior_MA + prior_MA_sigma, prior_MA + prior_MA_sigma], color="gray", alpha=0.2, label="GENIE Prior")

cov_MA, pred_MA, data, multisim_xs_MA_cov, xs_vars = get_MA_trio_cov_mat_pred(
    data_type = "NuWro",
)
constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(cov_MA, pred_MA, data)
extracted_MA = constrained_trio[0]
extracted_MA_sigma = np.sqrt(constrained_trio_cov[0][0])
plot_bold_MA_value(nuwro_fake_data_true_MA, extracted_MA, extracted_MA_sigma, "NuWro fake data", "k", 10)

for rw_true_MA in rw_MA_vals:
    reweighted_data = MA_reweighted_nuwro_fake_data_dic[rw_true_MA]
    constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(cov_MA, pred_MA, reweighted_data)
    extracted_MA = constrained_trio[0]
    extracted_MA_sigma = np.sqrt(constrained_trio_cov[0][0])
    plot_bold_MA_value(rw_true_MA, extracted_MA, extracted_MA_sigma, r"NuWro fake data with reweighted $M_A$", "tab:brown", 10)

plt.legend(loc="upper left", fontsize=6)
plt.xlabel(r"True $M_A$ (GeV/c$^2$)")
plt.ylabel(r"Reco $M_A$ Posterior (GeV/c$^2$)")
plt.xlim(0.6, 1.6)
plt.ylim(0.6, 1.6)
plt.title(r"NuWro fake data with reweighted $M_A$""\nno prior removal")
plt.savefig(filename + "_no_prior_removal.pdf")



In [ ]:
plt.figure(figsize=(10,7))
plt.plot([0, 2], [0, 2], c="k", ls="dashed", lw=2, zorder=5)

cov_MA, pred_MA, data, multisim_xs_MA_cov, xs_vars = get_MA_trio_cov_mat_pred(
    data_type = "NuWro",
)
constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(multisim_xs_MA_cov, pred_MA, data)
extracted_MA = prior_removed_MA
extracted_MA_sigma = prior_removed_MA_sigma
plot_bold_MA_value(nuwro_fake_data_true_MA, extracted_MA, extracted_MA_sigma, "NuWro fake data", "k", 10)

for rw_true_MA in rw_MA_vals:
    reweighted_data = MA_reweighted_nuwro_fake_data_dic[rw_true_MA]
    constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(multisim_xs_MA_cov, pred_MA, reweighted_data)
    extracted_MA = prior_removed_MA
    extracted_MA_sigma = prior_removed_MA_sigma
    plot_bold_MA_value(rw_true_MA, extracted_MA, extracted_MA_sigma, r"NuWro fake data with reweighted $M_A$", "tab:brown", 10)

plt.legend(loc="upper left", fontsize=6)
plt.xlabel(r"True $M_A$ (GeV/c$^2$)")
plt.ylabel(r"Reco $M_A$ (GeV/c$^2$)")
plt.xlim(0.6, 1.6)
plt.ylim(0.6, 1.6)
plt.title(r"NuWro fake data with reweighted $M_A$""\nmulti-sim XS only")
plt.savefig(filename + "_multisim_xs_only.pdf")

prior_MA = 1.1
prior_MA_sigma = 0.1
plt.figure(figsize=(10,7))
plt.plot([0, 2], [0, 2], c="k", ls="dashed", lw=2, zorder=5)
plt.fill_between([0, 2], [prior_MA - prior_MA_sigma, prior_MA - prior_MA_sigma], [prior_MA + prior_MA_sigma, prior_MA + prior_MA_sigma], color="gray", alpha=0.2, label="GENIE Prior")

cov_MA, pred_MA, data, multisim_xs_MA_cov, xs_vars = get_MA_trio_cov_mat_pred(
    data_type = "NuWro",
)
constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(multisim_xs_MA_cov, pred_MA, data)
extracted_MA = constrained_trio[0]
extracted_MA_sigma = np.sqrt(constrained_trio_cov[0][0])
plot_bold_MA_value(nuwro_fake_data_true_MA, extracted_MA, extracted_MA_sigma, "NuWro fake data", "k", 10)

for rw_true_MA in rw_MA_vals:
    reweighted_data = MA_reweighted_nuwro_fake_data_dic[rw_true_MA]
    constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(multisim_xs_MA_cov, pred_MA, reweighted_data)
    extracted_MA = constrained_trio[0]
    extracted_MA_sigma = np.sqrt(constrained_trio_cov[0][0])
    plot_bold_MA_value(rw_true_MA, extracted_MA, extracted_MA_sigma, r"NuWro fake data with reweighted $M_A$", "tab:brown", 10)

plt.legend(loc="upper left", fontsize=6)
plt.xlabel(r"True $M_A$ (GeV/c$^2$)")
plt.ylabel(r"Reco $M_A$ Posterior (GeV/c$^2$)")
plt.xlim(0.6, 1.6)
plt.ylim(0.6, 1.6)
plt.title(r"NuWro fake data with reweighted $M_A$""\nmulti-sim XS only, no prior removal")
plt.savefig(filename + "_multisim_xs_only_no_prior_removal.pdf")



In [ ]:
plt.figure(figsize=(10,7))
plt.plot([0, 2], [0, 2], c="k", ls="dashed", lw=2, zorder=5)

cov_MA, pred_MA, data, multisim_xs_MA_cov, xs_vars = get_MA_trio_cov_mat_pred(
    data_type = "NuWro",
    shape_type = "+100"
)
constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(cov_MA, pred_MA, data)
extracted_MA = prior_removed_MA
extracted_MA_sigma = prior_removed_MA_sigma
plot_bold_MA_value(nuwro_fake_data_true_MA, extracted_MA, extracted_MA_sigma, "NuWro fake data", "k", 10)

for rw_true_MA in rw_MA_vals:
    reweighted_data = MA_reweighted_nuwro_fake_data_dic[rw_true_MA]
    constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(cov_MA, pred_MA, reweighted_data)
    extracted_MA = prior_removed_MA
    extracted_MA_sigma = prior_removed_MA_sigma
    plot_bold_MA_value(rw_true_MA, extracted_MA, extracted_MA_sigma, r"NuWro fake data with reweighted $M_A$", "tab:brown", 10)

plt.legend(loc="upper left", fontsize=6)
plt.xlabel(r"True $M_A$ (GeV/c$^2$)")
plt.ylabel(r"Reco $M_A$ (GeV/c$^2$)")
plt.xlim(0.6, 1.6)
plt.ylim(0.6, 1.6)
plt.title(r"NuWro fake data with reweighted $M_A$""\nshape only (+100%)")
plt.savefig(filename + "_shape_only.pdf")

prior_MA = 1.1
prior_MA_sigma = 0.1
plt.figure(figsize=(10,7))
plt.plot([0, 2], [0, 2], c="k", ls="dashed", lw=2, zorder=5)
plt.fill_between([0, 2], [prior_MA - prior_MA_sigma, prior_MA - prior_MA_sigma], [prior_MA + prior_MA_sigma, prior_MA + prior_MA_sigma], color="gray", alpha=0.2, label="GENIE Prior")

cov_MA, pred_MA, data, multisim_xs_MA_cov, xs_vars = get_MA_trio_cov_mat_pred(
    data_type = "NuWro",
    shape_type = "+100"
)
constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(cov_MA, pred_MA, data)
extracted_MA = constrained_trio[0]
extracted_MA_sigma = np.sqrt(constrained_trio_cov[0][0])
plot_bold_MA_value(nuwro_fake_data_true_MA, extracted_MA, extracted_MA_sigma, "NuWro fake data", "k", 10)

for rw_true_MA in rw_MA_vals:
    reweighted_data = MA_reweighted_nuwro_fake_data_dic[rw_true_MA]
    constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(cov_MA, pred_MA, reweighted_data)
    extracted_MA = constrained_trio[0]
    extracted_MA_sigma = np.sqrt(constrained_trio_cov[0][0])
    plot_bold_MA_value(rw_true_MA, extracted_MA, extracted_MA_sigma, r"NuWro fake data with reweighted $M_A$", "tab:brown", 10)

plt.legend(loc="upper left", fontsize=6)
plt.xlabel(r"True $M_A$ (GeV/c$^2$)")
plt.ylabel(r"Reco $M_A$ Posterior (GeV/c$^2$)")
plt.xlim(0.6, 1.6)
plt.ylim(0.6, 1.6)
plt.title(r"NuWro fake data with reweighted $M_A$""\nshape only (+100%), no prior removal")
plt.savefig(filename + "_shape_only_no_prior_removal.pdf")



In [ ]:
plt.figure(figsize=(10,7))
plt.plot([0, 2], [0, 2], c="k", ls="dashed", lw=2, zorder=5)

cov_MA, pred_MA, data, multisim_xs_MA_cov, xs_vars = get_MA_trio_cov_mat_pred(
    data_type = "NuWro",
    shape_type = "+100"
)
constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(multisim_xs_MA_cov, pred_MA, data)
extracted_MA = prior_removed_MA
extracted_MA_sigma = prior_removed_MA_sigma
plot_bold_MA_value(nuwro_fake_data_true_MA, extracted_MA, extracted_MA_sigma, "NuWro fake data", "k", 10)

for rw_true_MA in rw_MA_vals:
    reweighted_data = MA_reweighted_nuwro_fake_data_dic[rw_true_MA]
    constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(multisim_xs_MA_cov, pred_MA, reweighted_data)
    extracted_MA = prior_removed_MA
    extracted_MA_sigma = prior_removed_MA_sigma
    plot_bold_MA_value(rw_true_MA, extracted_MA, extracted_MA_sigma, r"NuWro fake data with reweighted $M_A$", "tab:brown", 10)

plt.legend(loc="upper left", fontsize=6)
plt.xlabel(r"True $M_A$ (GeV/c$^2$)")
plt.ylabel(r"Reco $M_A$ (GeV/c$^2$)")
plt.xlim(0.6, 1.6)
plt.ylim(0.6, 1.6)
plt.title(r"NuWro fake data with reweighted $M_A$""\nmulti-sim XS only, shape only")
plt.savefig(filename + "_multisim_xs_only_shape_only.pdf")

prior_MA = 1.1
prior_MA_sigma = 0.1
plt.figure(figsize=(10,7))
plt.plot([0, 2], [0, 2], c="k", ls="dashed", lw=2, zorder=5)
plt.fill_between([0, 2], [prior_MA - prior_MA_sigma, prior_MA - prior_MA_sigma], [prior_MA + prior_MA_sigma, prior_MA + prior_MA_sigma], color="gray", alpha=0.2, label="GENIE Prior")

cov_MA, pred_MA, data, multisim_xs_MA_cov, xs_vars = get_MA_trio_cov_mat_pred(
    data_type = "NuWro",
    shape_type = "+100"
)
constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(multisim_xs_MA_cov, pred_MA, data)
extracted_MA = constrained_trio[0]
extracted_MA_sigma = np.sqrt(constrained_trio_cov[0][0])
plot_bold_MA_value(nuwro_fake_data_true_MA, extracted_MA, extracted_MA_sigma, "NuWro fake data", "k", 10)

for rw_true_MA in rw_MA_vals:
    reweighted_data = MA_reweighted_nuwro_fake_data_dic[rw_true_MA]
    constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(multisim_xs_MA_cov, pred_MA, reweighted_data)
    extracted_MA = constrained_trio[0]
    extracted_MA_sigma = np.sqrt(constrained_trio_cov[0][0])
    plot_bold_MA_value(rw_true_MA, extracted_MA, extracted_MA_sigma, r"NuWro fake data with reweighted $M_A$", "tab:brown", 10)

plt.legend(loc="upper left", fontsize=6)
plt.xlabel(r"True $M_A$ (GeV/c$^2$)")
plt.ylabel(r"Reco $M_A$ Posterior (GeV/c$^2$)")
plt.xlim(0.6, 1.6)
plt.ylim(0.6, 1.6)
plt.title(r"NuWro fake data with reweighted $M_A$""\nmulti-sim XS only, no prior removal, shape only")
plt.savefig(filename + "_multisim_xs_only_shape_only_no_prior_removal.pdf")



# M_A vs MEC Tests

In [ ]:
def get_ellipse_x_y_points_from_pair_cov(pair, cov, sigma=1):
    eigenvals, eigenvecs = np.linalg.eig(cov)
    eigenvecs = np.transpose(eigenvecs) 
    t = np.linspace(0, 2*np.pi, 100)
    ellipse_points_x = (np.cos(t) * sigma * np.sqrt(eigenvals[0]) * eigenvecs[0][1] + pair[1]
                    + np.sin(t) * sigma * np.sqrt(eigenvals[1]) * eigenvecs[1][1])
    ellpise_points_y = (np.cos(t) * sigma * np.sqrt(eigenvals[0]) * eigenvecs[0][0] + pair[0]
                    + np.sin(t) * sigma * np.sqrt(eigenvals[1]) * eigenvecs[1][0])
    return ellipse_points_x, ellpise_points_y

plt.figure(figsize=(10,7))
ax = plt.gca()
cov_MA, pred_MA, data, multisim_xs_MA_cov, xs_vars = get_MA_trio_cov_mat_pred(
    data_type = "NuWro",
)
constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(cov_MA, pred_MA, data)
extracted_MA = constrained_trio[0]
extracted_MA_sigma = np.sqrt(constrained_trio_cov[0][0])
extracted_MEC = constrained_trio[1]
extracted_MEC_sigma = np.sqrt(constrained_trio_cov[1][1])
#plt.errorbar(extracted_MA, extracted_MEC, xerr=extracted_MA_sigma, yerr=extracted_MEC_sigma, fmt="o", c="k", capsize=5, lw=2, capthick=2, label="NuWro fake data")
ellipse_points_y, ellipse_points_x = get_ellipse_x_y_points_from_pair_cov(constrained_trio, constrained_trio_cov)
plt.plot(ellipse_points_x, ellipse_points_y, c="k", lw=2, zorder=5, label="NuWro fake data")
plt.arrow(constrained_trio[0], constrained_trio[1], 1.03 - constrained_trio[0], extracted_MEC - constrained_trio[1], head_width=0.01, head_length=0.01, fc='k', ec='k')
for rw_true_MA in rw_MA_vals:
    reweighted_data = MA_reweighted_nuwro_fake_data_dic[rw_true_MA]
    constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(cov_MA, pred_MA, reweighted_data)
    extracted_MA = constrained_trio[0]
    extracted_MEC = constrained_trio[1]
    extracted_MEC_sigma = np.sqrt(constrained_trio_cov[1][1])
    label = ""
    if rw_true_MA == rw_MA_vals[0]:
        label = r"NuWro fake data with reweighted $M_A$"
    #plt.errorbar(extracted_MA, extracted_MEC, xerr=extracted_MA_sigma, yerr=extracted_MEC_sigma, fmt="o", c="tab:brown", capsize=5, lw=2, capthick=2, label=label)
    ellipse_points_y, ellipse_points_x = get_ellipse_x_y_points_from_pair_cov(constrained_trio, constrained_trio_cov)
    plt.plot(ellipse_points_x, ellipse_points_y, c="tab:brown", lw=2, zorder=5, label=label)
    plt.arrow(constrained_trio[0], constrained_trio[1], rw_true_MA - constrained_trio[0], extracted_MEC - constrained_trio[1], head_width=0.01, head_length=0.01, fc='tab:brown', ec='tab:brown')
plt.legend()
plt.xlabel(r"Extracted $M_A$ (GeV/c$^2$)")
plt.ylabel("Extracted NormCCMEC")
plt.title(r"NuWro fake data with reweighted $M_A$")
plt.savefig(filename + "_MA_MEC.pdf")

plt.figure(figsize=(10,7))
constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(multisim_xs_MA_cov, pred_MA, data)
extracted_MA = constrained_trio[0]
extracted_MA_sigma = np.sqrt(constrained_trio_cov[0][0])
extracted_MEC = constrained_trio[1]
extracted_MEC_sigma = np.sqrt(constrained_trio_cov[1][1])
#plt.errorbar(extracted_MA, extracted_MEC, xerr=extracted_MA_sigma, yerr=extracted_MEC_sigma, fmt="o", c="k", capsize=5, lw=2, capthick=2, label="NuWro fake data")
ellipse_points_y, ellipse_points_x = get_ellipse_x_y_points_from_pair_cov(constrained_trio, constrained_trio_cov)
plt.plot(ellipse_points_x, ellipse_points_y, c="k", lw=2, zorder=5, label="NuWro fake data")
plt.arrow(constrained_trio[0], constrained_trio[1], 1.03 - constrained_trio[0], extracted_MEC - constrained_trio[1], head_width=0.01, head_length=0.01, fc='k', ec='k')
for rw_true_MA in rw_MA_vals:
    reweighted_data = MA_reweighted_nuwro_fake_data_dic[rw_true_MA]
    constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(multisim_xs_MA_cov, pred_MA, reweighted_data)
    extracted_MA = constrained_trio[0]
    extracted_MEC = constrained_trio[1]
    extracted_MEC_sigma = np.sqrt(constrained_trio_cov[1][1])
    label = ""
    if rw_true_MA == rw_MA_vals[0]:
        label = r"NuWro fake data with reweighted $M_A$"
    #plt.errorbar(extracted_MA, extracted_MEC, xerr=extracted_MA_sigma, yerr=extracted_MEC_sigma, fmt="o", c="tab:brown", capsize=5, lw=2, capthick=2, label=label)
    ellipse_points_y, ellipse_points_x = get_ellipse_x_y_points_from_pair_cov(constrained_trio, constrained_trio_cov)
    plt.plot(ellipse_points_x, ellipse_points_y, c="tab:brown", lw=2, zorder=5, label=label)
    plt.arrow(constrained_trio[0], constrained_trio[1], rw_true_MA - constrained_trio[0], extracted_MEC - constrained_trio[1], head_width=0.01, head_length=0.01, fc='tab:brown', ec='tab:brown')
plt.legend()
plt.xlabel(r"Extracted $M_A$ (GeV/c$^2$)")
plt.ylabel("Extracted NormCCMEC")
plt.title(r"NuWro fake data with reweighted $M_A$""\nmulti-sim XS only")
plt.savefig(filename + "_MA_MEC_xs_only.pdf")



In [ ]:
plt.figure(figsize=(10,7))
cov_MA, pred_MA, data, multisim_xs_MA_cov, xs_vars = get_MA_trio_cov_mat_pred(
    data_type = "NuWro",
    shape_type = "+100"
)
constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(cov_MA, pred_MA, data)
extracted_MA = constrained_trio[0]
extracted_MA_sigma = np.sqrt(constrained_trio_cov[0][0])
extracted_MEC = constrained_trio[1]
extracted_MEC_sigma = np.sqrt(constrained_trio_cov[1][1])
#plt.errorbar(extracted_MA, extracted_MEC, xerr=extracted_MA_sigma, yerr=extracted_MEC_sigma, fmt="o", c="k", capsize=5, lw=2, capthick=2, label="NuWro fake data")
ellipse_points_y, ellipse_points_x = get_ellipse_x_y_points_from_pair_cov(constrained_trio, constrained_trio_cov)
plt.plot(ellipse_points_x, ellipse_points_y, c="k", lw=2, zorder=5, label="NuWro fake data")
plt.arrow(constrained_trio[0], constrained_trio[1], 1.03 - constrained_trio[0], extracted_MEC - constrained_trio[1], head_width=0.01, head_length=0.01, fc='k', ec='k')
for rw_true_MA in rw_MA_vals:
    reweighted_data = MA_reweighted_nuwro_fake_data_dic[rw_true_MA]
    constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(cov_MA, pred_MA, reweighted_data)
    extracted_MA = constrained_trio[0]
    extracted_MEC = constrained_trio[1]
    extracted_MEC_sigma = np.sqrt(constrained_trio_cov[1][1])
    label = ""
    if rw_true_MA == rw_MA_vals[0]:
        label = r"NuWro fake data with reweighted $M_A$"
    #plt.errorbar(extracted_MA, extracted_MEC, xerr=extracted_MA_sigma, yerr=extracted_MEC_sigma, fmt="o", c="tab:brown", capsize=5, lw=2, capthick=2, label=label)
    ellipse_points_y, ellipse_points_x = get_ellipse_x_y_points_from_pair_cov(constrained_trio, constrained_trio_cov)
    plt.plot(ellipse_points_x, ellipse_points_y, c="tab:brown", lw=2, zorder=5, label=label)
    plt.arrow(constrained_trio[0], constrained_trio[1], rw_true_MA - constrained_trio[0], extracted_MEC - constrained_trio[1], head_width=0.01, head_length=0.01, fc='tab:brown', ec='tab:brown')
plt.legend()
plt.xlabel(r"Extracted $M_A$ (GeV/c$^2$)")
plt.ylabel("Extracted NormCCMEC")
plt.title(r"NuWro fake data with reweighted $M_A$, shape only (+100%)")
plt.savefig(filename + "_MA_MEC_shape_only.pdf")

plt.figure(figsize=(10,7))
constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(multisim_xs_MA_cov, pred_MA, data)
extracted_MA = constrained_trio[0]
extracted_MA_sigma = np.sqrt(constrained_trio_cov[0][0])
extracted_MEC = constrained_trio[1]
extracted_MEC_sigma = np.sqrt(constrained_trio_cov[1][1])
#plt.errorbar(extracted_MA, extracted_MEC, xerr=extracted_MA_sigma, yerr=extracted_MEC_sigma, fmt="o", c="k", capsize=5, lw=2, capthick=2, label="NuWro fake data")
ellipse_points_y, ellipse_points_x = get_ellipse_x_y_points_from_pair_cov(constrained_trio, constrained_trio_cov)
plt.plot(ellipse_points_x, ellipse_points_y, c="k", lw=2, zorder=5, label="NuWro fake data")
plt.arrow(constrained_trio[0], constrained_trio[1], 1.03 - constrained_trio[0], extracted_MEC - constrained_trio[1], head_width=0.01, head_length=0.01, fc='k', ec='k')
for rw_true_MA in rw_MA_vals:
    reweighted_data = MA_reweighted_nuwro_fake_data_dic[rw_true_MA]
    constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(multisim_xs_MA_cov, pred_MA, reweighted_data)
    extracted_MA = constrained_trio[0]
    extracted_MEC = constrained_trio[1]
    extracted_MEC_sigma = np.sqrt(constrained_trio_cov[1][1])
    label = ""
    if rw_true_MA == rw_MA_vals[0]:
        label = r"NuWro fake data with reweighted $M_A$"
    #plt.errorbar(extracted_MA, extracted_MEC, xerr=extracted_MA_sigma, yerr=extracted_MEC_sigma, fmt="o", c="tab:brown", capsize=5, lw=2, capthick=2, label=label)
    ellipse_points_y, ellipse_points_x = get_ellipse_x_y_points_from_pair_cov(constrained_trio, constrained_trio_cov)
    plt.plot(ellipse_points_x, ellipse_points_y, c="tab:brown", lw=2, zorder=5, label=label)
    plt.arrow(constrained_trio[0], constrained_trio[1], rw_true_MA - constrained_trio[0], extracted_MEC - constrained_trio[1], head_width=0.01, head_length=0.01, fc='tab:brown', ec='tab:brown')
plt.legend()
plt.xlabel(r"Extracted $M_A$ (GeV/c$^2$)")
plt.ylabel("Extracted NormCCMEC")
plt.title(r"NuWro fake data with reweighted $M_A$""\nmulti-sim XS only, shape only (+100%)")
plt.savefig(filename + "_MA_MEC_xs_only_shape_only.pdf")



# Chi2 Tests

In [ ]:
from scipy.special import erfinv, erfcinv, erfc
from scipy.stats import chi2
from scipy.stats import poisson

def get_significance_from_p_value(p_value):
    sigma = np.sqrt(2.) * erfinv(1. - p_value)
    return sigma

def get_significance(chisquare, ndf):
    
    # probability of getting a more extreme result
    p_value = 1. - chi2.cdf(chisquare, ndf)
    
    sigma = np.sqrt(2.) * erfcinv(p_value)
    
    return p_value, sigma
    
def get_poisson_significance(measured, expected):
    
    # probability of getting an equal or more extreme result
    p_value = 1. - poisson.cdf(measured - 0.001, expected)
    
    sigma = np.sqrt(2.) * erfinv(1. - p_value)
    
    print(f"p value: {p_value}")
    print(f"significance: {sigma} sigma")


def get_p_value_from_significance(sigma):
    p_value = erfc(sigma / np.sqrt(2.))
    return p_value


In [ ]:
def get_chi2(cov, data, pred):
    return np.linalg.multi_dot([data - pred, np.linalg.inv(cov), data - pred])

def chi2_decomposition(cov, data, pred):

    N = len(data)

    eig_vals, eig_vecs = np.linalg.eig(cov)
    deltas = data - pred

    decomp_deltas = np.matmul(deltas, eig_vecs)

    chi2s_decomp = []
    local_p_values = []
    look_elsewhere_corrected_p_values = []
    local_signed_sigmas = []
    for eigenval_i in range(len(eig_vals)):

        eig_val = eig_vals[eigenval_i]

        decomp_delta = decomp_deltas[eigenval_i]

        local_signed_sigma = decomp_delta / np.sqrt(eig_val)
        local_chi2 = decomp_delta**2 / eig_val
        local_p_value, local_sigma = get_significance(local_chi2, 1)
        look_elsewhere_corrected_p_value = 1 - (1 - local_p_value)**N

        chi2s_decomp.append(local_chi2)
        local_p_values.append(local_p_value)
        local_signed_sigmas.append(local_signed_sigma)
        look_elsewhere_corrected_p_values.append(look_elsewhere_corrected_p_value)
    
    chi2_tot = np.sum(chi2s_decomp)
    
    global_p_value = np.min(look_elsewhere_corrected_p_values)
    global_sigma = get_significance_from_p_value(global_p_value)

    return chi2s_decomp, chi2_tot, local_signed_sigmas, global_p_value, global_sigma


In [ ]:
cov_MA, pred_MA, data, multisim_xs_MA_cov, xs_vars = get_MA_trio_cov_mat_pred(
    data_type = "NuWro",
)
true_MA = pred_MA[-3]
data = np.array(data)
pred_MA = np.array(pred_MA)
chi2_tot = get_chi2(cov_MA[:-3,:-3], data, pred_MA[:-3])
chi2_xs = get_chi2(multisim_xs_MA_cov[:-3,:-3], data, pred_MA[:-3])
p_value_tot, sigma_tot = get_significance(chi2_tot, 1152)
p_value_xs, sigma_xs = get_significance(chi2_xs, 1152)
print("total chi2, ndf, p value:", chi2_tot, 1152, p_value_tot)
print("xs chi2, ndf, p value:", chi2_xs, 1152, p_value_xs)
chi2s_decomp, chi2_tot, local_signed_sigmas, global_p_value, global_sigma = chi2_decomposition(cov_MA[:-3,:-3], data, pred_MA[:-3])
print(f"total chi2 decomposition: {chi2_tot}, global p value: {global_p_value}, global sigma: {global_sigma}")
xs_chi2s_decomp, xs_chi2_tot, xs_local_signed_sigmas, xs_global_p_value, xs_global_sigma = chi2_decomposition(multisim_xs_MA_cov[:-3,:-3], data, pred_MA[:-3])
print(f"xs chi2 decomposition: {xs_chi2_tot}, global p value: {xs_global_p_value}, global sigma: {xs_global_sigma}")

no_decomp_sigmas = []
xs_no_decomp_sigmas = []
total_global_sigmas = []
xs_global_sigmas = []
for rw_true_MA in rw_MA_vals:
    reweighted_data = MA_reweighted_nuwro_fake_data_dic[rw_true_MA]
    chi2_tot = get_chi2(cov_MA[:-3,:-3], reweighted_data, pred_MA[:-3])
    chi2_xs = get_chi2(multisim_xs_MA_cov[:-3,:-3], reweighted_data, pred_MA[:-3])
    curr_no_decomp_sigma = get_significance(chi2_tot, 1152)[1]
    curr_xs_no_decomp_sigma = get_significance(chi2_xs, 1152)[1]
    curr_chi2s_decomp, curr_chi2_tot, curr_local_sigmas, curr_global_p_value, curr_global_sigma = chi2_decomposition(cov_MA[:-3,:-3], reweighted_data, pred_MA[:-3])
    curr_xs_chi2s_decomp, curr_xs_chi2_tot, curr_xs_local_sigmas, curr_xs_global_p_value, curr_xs_global_sigma = chi2_decomposition(multisim_xs_MA_cov[:-3,:-3], reweighted_data, pred_MA[:-3])

    no_decomp_sigmas.append(curr_no_decomp_sigma)
    xs_no_decomp_sigmas.append(curr_xs_no_decomp_sigma)
    total_global_sigmas.append(curr_global_sigma)
    xs_global_sigmas.append(curr_xs_global_sigma)

plt.figure(figsize=(10,7))
x = np.linspace(0,len(chi2s_decomp)-1, 1001)
y1 = [-1,1]
y2 = [-2,2]
y3 = [-3,3]
plt.axhline(y3[0], color='red', alpha=0.2)
plt.axhline(y3[1], color='red', alpha=0.2)
plt.axhline(y2[0], color='orange', alpha=0.2)
plt.axhline(y2[1], color='orange', alpha=0.2)
plt.axhline(y1[0], color='green', alpha=0.2)
plt.axhline(y1[1], color='green', alpha=0.2)
plt.axhline(0, color='grey', alpha=0.2, ls='--')
plt.fill_between(x, y3[0], y2[0], color='red', alpha=0.2)
plt.fill_between(x, y2[0], y1[0], color='gold', alpha=0.2)
plt.fill_between(x, y1[0], y1[1], color='lime', alpha=0.2)
plt.fill_between(x, y1[1], y2[1], color='gold', alpha=0.2)
plt.fill_between(x, y2[1], y3[1], color='red', alpha=0.2)
plt.xlim(0,len(chi2s_decomp)-1)
plt.scatter(range(len(local_signed_sigmas)), local_signed_sigmas, label="Total Uncertainty")
plt.scatter(range(len(xs_local_signed_sigmas)), xs_local_signed_sigmas, label="Multi-sim XS Only")
plt.xlabel("Component")
plt.ylabel(r"Local $\sigma$")
plt.title(r"NuWro Fake Data $\chi^2$ Decomposition")
plt.legend()
plt.savefig(filename + "_chi2_decomp.pdf")

plt.figure(figsize=(10,7))
plt.scatter([], [], c="tab:blue", label=r"Normal $\sigma$")
plt.scatter([], [], c="tab:green", label=r"Normal $\sigma$, multi-sim XS only")
plt.scatter([], [], c="tab:red", label=r"Global $\sigma$")
plt.scatter([], [], c="tab:orange", label=r"Global $\sigma$, multi-sim XS only")
plt.scatter(true_MA, sigma_tot, c="tab:blue", marker="", s=400)
plt.scatter(true_MA, sigma_xs, c="tab:green", marker="+", s=400)
plt.scatter(true_MA, global_sigma, c="tab:red", marker="1", s=400)
plt.scatter(true_MA, xs_global_sigma, c="tab:orange", marker="2", s=400)
plt.scatter(rw_MA_vals, no_decomp_sigmas, c="tab:blue", marker="^", s=200)
plt.scatter(rw_MA_vals, xs_no_decomp_sigmas, c="tab:green", marker="v", s=200)
plt.scatter(rw_MA_vals, total_global_sigmas, c="tab:red", marker="^", s=200)
plt.scatter(rw_MA_vals, xs_global_sigmas, c="tab:orange", marker="v", s=200)
plt.legend()
plt.xlabel(r"True $M_A$ (GeV/c$^2$)")
plt.ylabel(r"$\sigma$")
plt.xlim(0.6, 1.6)
#plt.yscale("log")
plt.title(r"NuWro fake data with reweighted $M_A$")
plt.savefig(filename + "_chi2.pdf")


In [ ]:
cov_MA, pred_MA, data, multisim_xs_MA_cov, xs_vars = get_MA_trio_cov_mat_pred(
    data_type = "NuWro",
    shape_type = "+100"
)
true_MA = pred_MA[-3]
data = np.array(data)
pred_MA = np.array(pred_MA)
chi2_tot = get_chi2(cov_MA[:-3,:-3], data, pred_MA[:-3])
chi2_xs = get_chi2(multisim_xs_MA_cov[:-3,:-3], data, pred_MA[:-3])
p_value_tot, sigma_tot = get_significance(chi2_tot, 1152)
p_value_xs, sigma_xs = get_significance(chi2_xs, 1152)
print("total chi2, ndf, p value:", chi2_tot, 1152, p_value_tot)
print("xs chi2, ndf, p value:", chi2_xs, 1152, p_value_xs)
chi2s_decomp, chi2_tot, local_signed_sigmas, global_p_value, global_sigma = chi2_decomposition(cov_MA[:-3,:-3], data, pred_MA[:-3])
print(f"total chi2 decomposition: {chi2_tot}, global p value: {global_p_value}, global sigma: {global_sigma}")
xs_chi2s_decomp, xs_chi2_tot, xs_local_signed_sigmas, xs_global_p_value, xs_global_sigma = chi2_decomposition(multisim_xs_MA_cov[:-3,:-3], data, pred_MA[:-3])
print(f"xs chi2 decomposition: {xs_chi2_tot}, global p value: {xs_global_p_value}, global sigma: {xs_global_sigma}")

no_decomp_sigmas = []
xs_no_decomp_sigmas = []
total_global_sigmas = []
xs_global_sigmas = []
for rw_true_MA in rw_MA_vals:
    reweighted_data = MA_reweighted_nuwro_fake_data_dic[rw_true_MA]
    chi2_tot = get_chi2(cov_MA[:-3,:-3], reweighted_data, pred_MA[:-3])
    chi2_xs = get_chi2(multisim_xs_MA_cov[:-3,:-3], reweighted_data, pred_MA[:-3])
    curr_no_decomp_sigma = get_significance(chi2_tot, 1152)[1]
    curr_xs_no_decomp_sigma = get_significance(chi2_xs, 1152)[1]
    curr_chi2s_decomp, curr_chi2_tot, curr_local_sigmas, curr_global_p_value, curr_global_sigma = chi2_decomposition(cov_MA[:-3,:-3], reweighted_data, pred_MA[:-3])
    curr_xs_chi2s_decomp, curr_xs_chi2_tot, curr_xs_local_sigmas, curr_xs_global_p_value, curr_xs_global_sigma = chi2_decomposition(multisim_xs_MA_cov[:-3,:-3], reweighted_data, pred_MA[:-3])

    no_decomp_sigmas.append(curr_no_decomp_sigma)
    xs_no_decomp_sigmas.append(curr_xs_no_decomp_sigma)
    total_global_sigmas.append(curr_global_sigma)
    xs_global_sigmas.append(curr_xs_global_sigma)

plt.figure(figsize=(10,7))
x = np.linspace(0,len(chi2s_decomp)-1, 1001)
y1 = [-1,1]
y2 = [-2,2]
y3 = [-3,3]
plt.axhline(y3[0], color='red', alpha=0.2)
plt.axhline(y3[1], color='red', alpha=0.2)
plt.axhline(y2[0], color='orange', alpha=0.2)
plt.axhline(y2[1], color='orange', alpha=0.2)
plt.axhline(y1[0], color='green', alpha=0.2)
plt.axhline(y1[1], color='green', alpha=0.2)
plt.axhline(0, color='grey', alpha=0.2, ls='--')
plt.fill_between(x, y3[0], y2[0], color='red', alpha=0.2)
plt.fill_between(x, y2[0], y1[0], color='gold', alpha=0.2)
plt.fill_between(x, y1[0], y1[1], color='lime', alpha=0.2)
plt.fill_between(x, y1[1], y2[1], color='gold', alpha=0.2)
plt.fill_between(x, y2[1], y3[1], color='red', alpha=0.2)
plt.xlim(0,len(chi2s_decomp)-1)
plt.scatter(range(len(local_signed_sigmas)), local_signed_sigmas, label="Total Uncertainty")
plt.scatter(range(len(xs_local_signed_sigmas)), xs_local_signed_sigmas, label="Multi-sim XS Only")
plt.xlabel("Component")
plt.ylabel(r"Local $\sigma$")
plt.title(r"NuWro Fake Data $\chi^2$ Decomposition""\nshape only (+100%)")
plt.legend()
plt.savefig(filename + "_chi2_decomp_shape_only.pdf")

plt.figure(figsize=(10,7))
plt.scatter([], [], c="tab:blue", label=r"Normal $\sigma$")
plt.scatter([], [], c="tab:green", label=r"Normal $\sigma$, multi-sim XS only")
plt.scatter([], [], c="tab:red", label=r"Global $\sigma$")
plt.scatter([], [], c="tab:orange", label=r"Global $\sigma$, multi-sim XS only")
plt.scatter(true_MA, sigma_tot, c="tab:blue", marker="", s=400)
plt.scatter(true_MA, sigma_xs, c="tab:green", marker="+", s=400)
plt.scatter(true_MA, global_sigma, c="tab:red", marker="1", s=400)
plt.scatter(true_MA, xs_global_sigma, c="tab:orange", marker="2", s=400)
plt.scatter(rw_MA_vals, no_decomp_sigmas, c="tab:blue", marker="^", s=200)
plt.scatter(rw_MA_vals, xs_no_decomp_sigmas, c="tab:green", marker="v", s=200)
plt.scatter(rw_MA_vals, total_global_sigmas, c="tab:red", marker="^", s=200)
plt.scatter(rw_MA_vals, xs_global_sigmas, c="tab:orange", marker="v", s=200)
plt.legend()
plt.xlabel(r"True $M_A$ (GeV/c$^2$)")
plt.ylabel(r"$\sigma$")
plt.xlim(0.6, 1.6)
#plt.yscale("log")
plt.title(r"NuWro fake data with reweighted $M_A$""\nshape only (+100%)")
plt.savefig(filename + "_chi2_shape_only.pdf")


# Collapsing E_nu

In [ ]:
from get_MA_trio_cov_mat import collapse_4D_to_2D, collapse_4D_to_1D

plt.figure(figsize=(10,7))
plt.plot([0, 2], [0, 2], c="k", ls="dashed", lw=2, zorder=5)
cov_MA, pred_MA, data, multisim_xs_MA_cov, xs_vars = get_MA_trio_cov_mat_pred(
    data_type = "NuWro",
    collapse_type = "2D"
)
constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(cov_MA, pred_MA, data)
extracted_MA = prior_removed_MA
extracted_MA_sigma = prior_removed_MA_sigma
plot_bold_MA_value(nuwro_fake_data_true_MA, extracted_MA, extracted_MA_sigma, "NuWro fake data", "k", 10)
for rw_true_MA in rw_MA_vals:
    reweighted_data = collapse_4D_to_2D(MA_reweighted_nuwro_fake_data_dic[rw_true_MA])
    constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(cov_MA, pred_MA, reweighted_data)
    extracted_MA = prior_removed_MA
    extracted_MA_sigma = prior_removed_MA_sigma
    plot_bold_MA_value(rw_true_MA, extracted_MA, extracted_MA_sigma, r"NuWro fake data with reweighted $M_A$", "tab:brown", 10)
plt.legend(loc="upper left", fontsize=6)
plt.xlabel(r"True $M_A$ (GeV/c$^2$)")
plt.ylabel(r"Reco $M_A$ (GeV/c$^2$)")
plt.xlim(0.6, 1.6)
plt.ylim(0.6, 1.6)
plt.title(r"NuWro fake data with reweighted $M_A$""\nCollapsed to 2D muon momentum and angle")
plt.savefig(filename + "_2D.pdf")


plt.figure(figsize=(10,7))
plt.plot([0, 2], [0, 2], c="k", ls="dashed", lw=2, zorder=5)
cov_MA, pred_MA, data, multisim_xs_MA_cov, xs_vars = get_MA_trio_cov_mat_pred(
    data_type = "NuWro",
    collapse_type = "1D"
)
constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(cov_MA, pred_MA, data)
extracted_MA = prior_removed_MA
extracted_MA_sigma = prior_removed_MA_sigma
plot_bold_MA_value(nuwro_fake_data_true_MA, extracted_MA, extracted_MA_sigma, "NuWro fake data", "k", 10)
for rw_true_MA in rw_MA_vals:
    reweighted_data = collapse_4D_to_1D(MA_reweighted_nuwro_fake_data_dic[rw_true_MA])
    constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(cov_MA, pred_MA, reweighted_data)
    extracted_MA = prior_removed_MA
    extracted_MA_sigma = prior_removed_MA_sigma
    plot_bold_MA_value(rw_true_MA, extracted_MA, extracted_MA_sigma, r"NuWro fake data with reweighted $M_A$", "tab:brown", 10)
plt.legend(loc="upper left", fontsize=6)
plt.xlabel(r"True $M_A$ (GeV/c$^2$)")
plt.ylabel(r"Reco $M_A$ (GeV/c$^2$)")
plt.xlim(0.6, 1.6)
plt.ylim(0.6, 1.6)
plt.title(r"NuWro fake data with reweighted $M_A$""\nCollapsed to 1D muon momentum")
plt.savefig(filename + "_1D.pdf")

In [ ]:
cov_MA, pred_MA, data, multisim_xs_MA_cov, xs_vars = get_MA_trio_cov_mat_pred(
    data_type = "NuWro",
    collapse_type = "2D"
)
true_MA = pred_MA[-3]
data = np.array(data)
pred_MA = np.array(pred_MA)
chi2_tot = get_chi2(cov_MA[:-3,:-3], data, pred_MA[:-3])
chi2_xs = get_chi2(multisim_xs_MA_cov[:-3,:-3], data, pred_MA[:-3])
p_value_tot, sigma_tot = get_significance(chi2_tot, 1152)
p_value_xs, sigma_xs = get_significance(chi2_xs, 1152)
print("total chi2, ndf, p value:", chi2_tot, 1152, p_value_tot)
print("xs chi2, ndf, p value:", chi2_xs, 1152, p_value_xs)
chi2s_decomp, chi2_tot, local_signed_sigmas, global_p_value, global_sigma = chi2_decomposition(cov_MA[:-3,:-3], data, pred_MA[:-3])
print(f"total chi2 decomposition: {chi2_tot}, global p value: {global_p_value}, global sigma: {global_sigma}")
xs_chi2s_decomp, xs_chi2_tot, xs_local_signed_sigmas, xs_global_p_value, xs_global_sigma = chi2_decomposition(multisim_xs_MA_cov[:-3,:-3], data, pred_MA[:-3])
print(f"xs chi2 decomposition: {xs_chi2_tot}, global p value: {xs_global_p_value}, global sigma: {xs_global_sigma}")

no_decomp_sigmas = []
xs_no_decomp_sigmas = []
total_global_sigmas = []
xs_global_sigmas = []
for rw_true_MA in rw_MA_vals:
    reweighted_data = collapse_4D_to_2D(MA_reweighted_nuwro_fake_data_dic[rw_true_MA])
    chi2_tot = get_chi2(cov_MA[:-3,:-3], reweighted_data, pred_MA[:-3])
    chi2_xs = get_chi2(multisim_xs_MA_cov[:-3,:-3], reweighted_data, pred_MA[:-3])
    curr_no_decomp_sigma = get_significance(chi2_tot, 1152)[1]
    curr_xs_no_decomp_sigma = get_significance(chi2_xs, 1152)[1]
    curr_chi2s_decomp, curr_chi2_tot, curr_local_sigmas, curr_global_p_value, curr_global_sigma = chi2_decomposition(cov_MA[:-3,:-3], reweighted_data, pred_MA[:-3])
    curr_xs_chi2s_decomp, curr_xs_chi2_tot, curr_xs_local_sigmas, curr_xs_global_p_value, curr_xs_global_sigma = chi2_decomposition(multisim_xs_MA_cov[:-3,:-3], reweighted_data, pred_MA[:-3])

    no_decomp_sigmas.append(curr_no_decomp_sigma)
    xs_no_decomp_sigmas.append(curr_xs_no_decomp_sigma)
    total_global_sigmas.append(curr_global_sigma)
    xs_global_sigmas.append(curr_xs_global_sigma)

plt.figure(figsize=(10,7))
x = np.linspace(0,len(chi2s_decomp)-1, 1001)
y1 = [-1,1]
y2 = [-2,2]
y3 = [-3,3]
plt.axhline(y3[0], color='red', alpha=0.2)
plt.axhline(y3[1], color='red', alpha=0.2)
plt.axhline(y2[0], color='orange', alpha=0.2)
plt.axhline(y2[1], color='orange', alpha=0.2)
plt.axhline(y1[0], color='green', alpha=0.2)
plt.axhline(y1[1], color='green', alpha=0.2)
plt.axhline(0, color='grey', alpha=0.2, ls='--')
plt.fill_between(x, y3[0], y2[0], color='red', alpha=0.2)
plt.fill_between(x, y2[0], y1[0], color='gold', alpha=0.2)
plt.fill_between(x, y1[0], y1[1], color='lime', alpha=0.2)
plt.fill_between(x, y1[1], y2[1], color='gold', alpha=0.2)
plt.fill_between(x, y2[1], y3[1], color='red', alpha=0.2)
plt.xlim(0,len(chi2s_decomp)-1)
plt.scatter(range(len(local_signed_sigmas)), local_signed_sigmas, label="Total Uncertainty")
plt.scatter(range(len(xs_local_signed_sigmas)), xs_local_signed_sigmas, label="Multi-sim XS Only")
plt.xlabel("Component")
plt.ylabel(r"Local $\sigma$")
plt.title(r"NuWro Fake Data $\chi^2$ Decomposition""\nCollapsed to 2D muon momentum and angle")
plt.legend()
plt.savefig(filename + "_chi2_decomp_2D.pdf")

plt.figure(figsize=(10,7))
plt.scatter([], [], c="tab:blue", label=r"Normal $\sigma$")
plt.scatter([], [], c="tab:green", label=r"Normal $\sigma$, multi-sim XS only")
plt.scatter([], [], c="tab:red", label=r"Global $\sigma$")
plt.scatter([], [], c="tab:orange", label=r"Global $\sigma$, multi-sim XS only")
plt.scatter(true_MA, sigma_tot, c="tab:blue", marker="", s=400)
plt.scatter(true_MA, sigma_xs, c="tab:green", marker="+", s=400)
plt.scatter(true_MA, global_sigma, c="tab:red", marker="1", s=400)
plt.scatter(true_MA, xs_global_sigma, c="tab:orange", marker="2", s=400)
plt.scatter(rw_MA_vals, no_decomp_sigmas, c="tab:blue", marker="^", s=200)
plt.scatter(rw_MA_vals, xs_no_decomp_sigmas, c="tab:green", marker="v", s=200)
plt.scatter(rw_MA_vals, total_global_sigmas, c="tab:red", marker="^", s=200)
plt.scatter(rw_MA_vals, xs_global_sigmas, c="tab:orange", marker="v", s=200)
plt.legend()
plt.xlabel(r"True $M_A$ (GeV/c$^2$)")
plt.ylabel(r"$\sigma$")
plt.xlim(0.6, 1.6)
#plt.yscale("log")
plt.title(r"NuWro fake data with reweighted $M_A$""\nCollapsed to 2D muon momentum and angle")
plt.savefig(filename + "_chi2_2D.pdf")


In [ ]:
cov_MA, pred_MA, data, multisim_xs_MA_cov, xs_vars = get_MA_trio_cov_mat_pred(
    data_type = "NuWro",
    collapse_type = "1D"
)
true_MA = pred_MA[-3]
data = np.array(data)
pred_MA = np.array(pred_MA)
chi2_tot = get_chi2(cov_MA[:-3,:-3], data, pred_MA[:-3])
chi2_xs = get_chi2(multisim_xs_MA_cov[:-3,:-3], data, pred_MA[:-3])
p_value_tot, sigma_tot = get_significance(chi2_tot, 1152)
p_value_xs, sigma_xs = get_significance(chi2_xs, 1152)
print("total chi2, ndf, p value:", chi2_tot, 1152, p_value_tot)
print("xs chi2, ndf, p value:", chi2_xs, 1152, p_value_xs)
chi2s_decomp, chi2_tot, local_signed_sigmas, global_p_value, global_sigma = chi2_decomposition(cov_MA[:-3,:-3], data, pred_MA[:-3])
print(f"total chi2 decomposition: {chi2_tot}, global p value: {global_p_value}, global sigma: {global_sigma}")
xs_chi2s_decomp, xs_chi2_tot, xs_local_signed_sigmas, xs_global_p_value, xs_global_sigma = chi2_decomposition(multisim_xs_MA_cov[:-3,:-3], data, pred_MA[:-3])
print(f"xs chi2 decomposition: {xs_chi2_tot}, global p value: {xs_global_p_value}, global sigma: {xs_global_sigma}")

no_decomp_sigmas = []
xs_no_decomp_sigmas = []
total_global_sigmas = []
xs_global_sigmas = []
for rw_true_MA in rw_MA_vals:
    reweighted_data = collapse_4D_to_1D(MA_reweighted_nuwro_fake_data_dic[rw_true_MA])
    chi2_tot = get_chi2(cov_MA[:-3,:-3], reweighted_data, pred_MA[:-3])
    chi2_xs = get_chi2(multisim_xs_MA_cov[:-3,:-3], reweighted_data, pred_MA[:-3])
    curr_no_decomp_sigma = get_significance(chi2_tot, 1152)[1]
    curr_xs_no_decomp_sigma = get_significance(chi2_xs, 1152)[1]
    curr_chi2s_decomp, curr_chi2_tot, curr_local_sigmas, curr_global_p_value, curr_global_sigma = chi2_decomposition(cov_MA[:-3,:-3], reweighted_data, pred_MA[:-3])
    curr_xs_chi2s_decomp, curr_xs_chi2_tot, curr_xs_local_sigmas, curr_xs_global_p_value, curr_xs_global_sigma = chi2_decomposition(multisim_xs_MA_cov[:-3,:-3], reweighted_data, pred_MA[:-3])

    no_decomp_sigmas.append(curr_no_decomp_sigma)
    xs_no_decomp_sigmas.append(curr_xs_no_decomp_sigma)
    total_global_sigmas.append(curr_global_sigma)
    xs_global_sigmas.append(curr_xs_global_sigma)

plt.figure(figsize=(10,7))
x = np.linspace(0,len(chi2s_decomp)-1, 1001)
y1 = [-1,1]
y2 = [-2,2]
y3 = [-3,3]
plt.axhline(y3[0], color='red', alpha=0.2)
plt.axhline(y3[1], color='red', alpha=0.2)
plt.axhline(y2[0], color='orange', alpha=0.2)
plt.axhline(y2[1], color='orange', alpha=0.2)
plt.axhline(y1[0], color='green', alpha=0.2)
plt.axhline(y1[1], color='green', alpha=0.2)
plt.axhline(0, color='grey', alpha=0.2, ls='--')
plt.fill_between(x, y3[0], y2[0], color='red', alpha=0.2)
plt.fill_between(x, y2[0], y1[0], color='gold', alpha=0.2)
plt.fill_between(x, y1[0], y1[1], color='lime', alpha=0.2)
plt.fill_between(x, y1[1], y2[1], color='gold', alpha=0.2)
plt.fill_between(x, y2[1], y3[1], color='red', alpha=0.2)
plt.xlim(0,len(chi2s_decomp)-1)
plt.scatter(range(len(local_signed_sigmas)), local_signed_sigmas, label="Total Uncertainty")
plt.scatter(range(len(xs_local_signed_sigmas)), xs_local_signed_sigmas, label="Multi-sim XS Only")
plt.xlabel("Component")
plt.ylabel(r"Local $\sigma$")
plt.title(r"NuWro Fake Data $\chi^2$ Decomposition""\nCollapsed to 1D muon momentum")
plt.legend()
plt.savefig(filename + "_chi2_decomp_1D.pdf")

plt.figure(figsize=(10,7))
plt.scatter([], [], c="tab:blue", label=r"Normal $\sigma$")
plt.scatter([], [], c="tab:green", label=r"Normal $\sigma$, multi-sim XS only")
plt.scatter([], [], c="tab:red", label=r"Global $\sigma$")
plt.scatter([], [], c="tab:orange", label=r"Global $\sigma$, multi-sim XS only")
plt.scatter(true_MA, sigma_tot, c="tab:blue", marker="", s=400)
plt.scatter(true_MA, sigma_xs, c="tab:green", marker="+", s=400)
plt.scatter(true_MA, global_sigma, c="tab:red", marker="1", s=400)
plt.scatter(true_MA, xs_global_sigma, c="tab:orange", marker="2", s=400)
plt.scatter(rw_MA_vals, no_decomp_sigmas, c="tab:blue", marker="^", s=200)
plt.scatter(rw_MA_vals, xs_no_decomp_sigmas, c="tab:green", marker="v", s=200)
plt.scatter(rw_MA_vals, total_global_sigmas, c="tab:red", marker="^", s=200)
plt.scatter(rw_MA_vals, xs_global_sigmas, c="tab:orange", marker="v", s=200)
plt.legend()
plt.xlabel(r"True $M_A$ (GeV/c$^2$)")
plt.ylabel(r"$\sigma$")
plt.xlim(0.6, 1.6)
#plt.yscale("log")
plt.title(r"NuWro fake data with reweighted $M_A$""\nCollapsed to 1D muon momentum")
plt.savefig(filename + "_chi2_1D.pdf")


# Normalization Test

In [ ]:
plt.figure(figsize=(10,7))

plt.figure(figsize=(10,7))
for rw_true_MA in [0.7, 0.8, 0.9, 1.0, 1.03, 1.1, 1.2, 1.3, 1.4, 1.5]:
    plt.scatter(rw_true_MA, np.sum(MA_reweighted_nuwro_fake_data_dic[rw_true_MA]), c="tab:blue", s=100)
plt.scatter(1.03, np.sum(data), c="k", s=100)
plt.xlabel("True $M_A$ (GeV/c$^2$)")
plt.ylabel("Number of fake data events")
plt.show()
